## The aim of this notebook is to provide insight and answers to the following business questions

For queries involving transactions, you may need the following formula for calculated fields:
GrossProfit = PurchasePrice - WholesaleCost

1.How many customers are subscribed to the email list? How many are not?

2.Create a query that returns all of the root beer brand names that use either cane sugar or honey as a sweetener as well as all root beer brands headquartered in California (regardless of sweetener).

3.Which customer has the most root beer reviews? Write a query that lists first and last name and Review Count (the number of reviews for each customer). Sort by Review Count in descending order. 

4.Are certain zip codes stingier in their reviews of root beers? Create a query that calculates the average star rating by zip code. Include zip code and AvgStars in your results Sort in descending order by AvgStars. What zip code (or zip codes) give the lowest rating? What zip code (or zip codes gives the highest?

5.Are men or women providing the biggest share of our revenue? Create a query that lists revenue by gender. Hint: You will need purchaseprice and wholesalecost to calculate revenue.

6.How many of our customers made their first purchase in the year 2012?

7.Are there any root beer brands that don't have a website listed in the database?

8.Dr. Wells is very interested in how the different brands are performing. Specifically, he wants to know how many bottles (not cans) of each root beer brand have been sold in total. He only wants bottles from brands that are “currently offered” in the Root box kiosks. Create a query that lists two fields: Brand Name (which is the brand name of the Root Beer) and NumberOfBottles (which is the total number of bottles of currently offered root beer sold since the company began). Sort this list in descending order by NumberOfBottles.

9.A few weeks ago, Dr. Wells helped a customer who had a problem with a transaction. Dr. Wells needs to contact this customer to follow up, but unfortunately spilled all over the paper with the customer’s name and contact information. The only portion of the notes that are readable says “4509” which Dr. Wells thinks is part of a credit card number. Create a query with two fields: Name (A concatenation of the customer’s first name and last name separated by a space, e.g., “John Gregglan” but without the quotes) and Phone Number (the customer’s phone number). Hint: You will need to use the aggregate function “Group By”. Make sure not to display the sensitive credit card number.


In [1]:
import pandas as pd
import numpy as np

In [2]:
customer_df = pd.read_csv('customer.csv')
geolocation_df = pd.read_csv('geolocation.csv')
location_df = pd.read_csv('location.csv')
rootbeer_df = pd.read_csv('rootbeer.csv')
rootbeerbrand_df = pd.read_csv('rootbeerbrand.csv')
rootbeerreview_df = pd.read_csv('rootbeerreview.csv')
transaction_df = pd.read_csv('transaction.csv')

In [3]:
print('The shape of customer dataframe is : ', customer_df.shape)
print('The shape of geolocation dataframe is : ', geolocation_df.shape)
print('The shape of location dataframe is : ', location_df.shape)
print('The shape of rootbeer dataframe is : ', rootbeer_df.shape)
print('The shape of rootbeerbrand dataframe is : ', rootbeerbrand_df.shape)
print('The shape of rootbeerreview dataframe is : ', rootbeerreview_df.shape)
print('The shape of transaction dataframe is : ', transaction_df.shape)

The shape of customer dataframe is :  (554, 12)
The shape of geolocation dataframe is :  (3, 4)
The shape of location dataframe is :  (3, 6)
The shape of rootbeer dataframe is :  (6430, 5)
The shape of rootbeerbrand dataframe is :  (24, 22)
The shape of rootbeerreview dataframe is :  (714, 5)
The shape of transaction dataframe is :  (6312, 8)


In [4]:
customer_df.head()

,customerid,firstname,lastname,streetaddress,city,state,zipcode,email,phonenumber,firstpurchasedate,subscribedtoemaillist,gender
0,101811,Kenneth,Walton,6715 Commonwealth Dr,Sacramento,CA,94256,walton.k76@fastmail.com,(916) 918-1561,2013-05-30,False,M
1,103508,Madeleine,Jones,3603 Leola Way,Sacramento,CA,94258,j_madeleine@gmail.com,(916) 186-9423,2013-02-06,False,F
2,104939,Damen,Wheeler,6740 Branwood Way,Orangevale,CA,95662,dwheeler@outlook.com,(916) 164-1156,2013-04-11,False,M
3,105549,Kevin,Gilbert,3198 Livingston Way,Folsom,CA,95671,kgilbert@fastmail.com,(916) 304-9859,2013-02-28,True,M
4,105771,John,Young,663 Westward Way,Sacramento,CA,95899,john.y90@mail.com,(916) 730-6109,2013-09-05,True,M


In [5]:
geolocation_df.head()

,locationid,latitude,longitude,location
0,0,0.000000,0.000000,POINT(0 0)
1,1,38.566129,-121.426432,POINT(-121.426432 38.566129)
2,2,38.559615,-121.422430,POINT(-121.42243 38.559615)


In [6]:
location_df.head()

,locationid,locationname,streetaddress,city,state,zipcode
0,0,LOST,NaN,NaN,NaN,NaN
1,1,Sac State American River Courtyard,6000 J St,Sacramento,CA,95819.0
2,2,Sac State Union,6000 J St,Sacramento,CA,95819.0


In [7]:
rootbeer_df.head()

,rootbeerid,brandid,containertype,locationid,purchasedate
0,100054,10001,Bottle,2,2014-07-01
1,100081,10001,Can,2,2014-07-14
2,100019,10001,Can,2,2014-08-12
3,100080,10001,Can,2,2014-08-24
4,100135,10001,Bottle,2,2014-09-11


In [8]:
rootbeerbrand_df.head()

,brandid,brandname,firstbrewedyear,breweryname,city,state,country,description,canesugar,cornsyrup,...,caffeinated,alcoholic,availableincans,availableinbottles,availableinkegs,website,facebookpage,twitter,wholesalecost,currentretailprice
0,10001,A&W,1919,Dr Pepper Snapple Group,Lodi,CA,United States,After setting up the first A&W Root Beer stand...,False,True,...,False,False,True,False,False,http://www.rootbeer.com/,NaN,NaN,0.42,1
1,10002,A.J. Stephans,1926,AJ Stephans Beverages,Fall River,MA,United States,AJ Stephans Company makes \nthe finest elixirs...,True,False,...,False,False,False,True,False,http://www.ajstephans.com/,NaN,NaN,0.98,3
2,10003,Abita,1986,Abita Brewery,Covington,LA,United States,Abita Root Beer is made with a hot mix process...,True,False,...,False,False,False,True,False,https://abita.com/brews/our_brews/abita-root-beer,NaN,NaN,1.13,3
3,10004,Barq's,1898,Coca-Cola,New Orleans,LA,United States,Since 1898 Barq's root beer has had a simple s...,False,True,...,True,False,True,False,False,http://www.barqs.com/,NaN,NaN,0.40,1
4,10005,Bedfords,1984,Northwest Soda Works,Port Angeles,WA,United States,"Always ice cold, “never with ice”.",True,False,...,False,False,False,True,False,http://bedfordssodas.com/products.html,NaN,NaN,1.10,3


In [9]:
rootbeerreview_df.head()

,customerid,brandid,starrating,reviewdate,review
0,101811,10012,5,2013-07-15,NaN
1,101811,10014,1,2013-07-08,NaN
2,101811,10015,3,2013-07-25,NaN
3,101811,10021,2,2013-11-15,NaN
4,105549,10015,2,2013-08-11,NaN


In [10]:
transaction_df.head()

,transactionid,creditcardnumber,customerid,transactiondate,creditcardtype,locationid,rootbeerid,purchaseprice
0,100000,6011583832864739,864896,2014-07-07,Discover,2,105661,3
1,100001,6011583832864739,864896,2014-07-07,Discover,2,105798,3
2,100002,6011583832864739,864896,2014-07-07,Discover,2,102514,3
3,100003,6011583832864739,864896,2014-07-07,Discover,2,105623,3
4,100004,4716634257568793,610766,2014-07-13,Visa,1,103940,3


#### 1.How many customers are subscribed to the email list? How many are not?

In [11]:
sub = customer_df['subscribedtoemaillist']== True
print('The number of customers subscribed to email are : ', customer_df[sub].shape[0])
print('The number of customers NOT subscribed to email are : ', customer_df[-sub].shape[0])

The number of customers subscribed to email are :  299
The number of customers NOT subscribed to email are :  255


#### 2.Create a query that returns all of the root beer brand names that use either cane sugar or honey as a sweetener as well as all root beer brands headquartered in California (regardless of sweetener).

In [12]:
filt_sweet = (rootbeerbrand_df['canesugar'] == True) |  (rootbeerbrand_df['honey'] == True)
brd_name_sugar_or_honey = rootbeerbrand_df[filt_sweet]['brandname']
brd_name_sugar_or_honey

1            A.J. Stephans
2                    Abita
4                 Bedfords
5                  Bulldog
6                Bundaberg
7            Captain Eli's
8                    Dad's
10                Dominion
11                  Fitz's
12    Sparky's Fresh Draft
13                   Gales
14        Henry Weinhard's
15                     IBC
17              River City
18                 Frostie
19                Sprecher
20           Thomas Kemper
21                Virgil's
23                    1919
Name: brandname, dtype: object

In [13]:
filt_loc = rootbeerbrand_df['state'] == 'CA'
located_CA = rootbeerbrand_df[filt_loc]['brandname']
located_CA 

0                      A&W
5                  Bulldog
12    Sparky's Fresh Draft
16                     Mug
17              River City
Name: brandname, dtype: object

#### 3.Which customer has the most root beer reviews? Write a query that lists first and last name and Review Count (the number of reviews for each customer). Sort by Review Count in descending order.

In [14]:
test = rootbeerreview_df[rootbeerreview_df['review'].notna()].groupby(['customerid']).agg({'review' : 'count'}).reset_index()
test = pd.merge(customer_df[['customerid','firstname', 'lastname']], test, on = 'customerid', how = 'right').sort_values(by = 'review', ascending = False)
test

,customerid,firstname,lastname,review
5,170500,Violet,Ramey,2
12,331115,Janet,Koons,2
0,105771,John,Young,1
1,122818,Beatrice,Abramson,1
2,127472,George,Stanich,1
3,158310,Doris,Garrett,1
4,168607,Angela,Metzger,1
6,185863,Constance,Gonzalez,1
7,191050,Charlene,Boss,1
8,216081,Jeff,Float,1


In [15]:
# customer with the most reviews
test[test['review'] == test['review'].max()]

,customerid,firstname,lastname,review
5,170500,Violet,Ramey,2
12,331115,Janet,Koons,2


#### 4.Are certain zip codes stingier in their reviews of root beers? Create a query that calculates the average star rating by zip code. Include zip code and AvgStars in your results Sort in descending order by AvgStars. What zip code (or zip codes) give the lowest rating? What zip code (or zip codes gives the highest?

In [16]:
new_df = pd.merge(customer_df, rootbeerreview_df, how = 'right', on = 'customerid')
news_df = pd.DataFrame(new_df, columns=['zipcode','starrating'])
zip_group= news_df.groupby(['zipcode'])
avgStar_rating = pd.DataFrame({'AvgStars' : zip_group['starrating'].mean().sort_values(ascending = False)})
print('The overall avg starrating is',avgStar_rating.mean(),'. Hence avg rating lower that the avg starrating are considered low and anything above as high')

The overall avg starrating is AvgStars    3.007223
dtype: float64 . Hence avg rating lower that the avg starrating are considered low and anything above as high


In [17]:
avgStar_rating.reset_index(inplace = True)
avgStar_rating

,zipcode,AvgStars
0,94211,4.333333
1,95830,4.285714
2,95819,4.142857
3,95680,4.111111
4,95814,4.000000
...,...,...
121,95841,1.500000
122,95671,1.500000
123,94268,1.500000
124,95693,1.000000


In [18]:
ratings_df = avgStar_rating['AvgStars'] >= avgStar_rating['AvgStars'].mean()
Highest_rating = avgStar_rating[ratings_df]
lowest_rating = avgStar_rating[-ratings_df]

In [19]:
Highest_rating

,zipcode,AvgStars
0,94211,4.333333
1,95830,4.285714
2,95819,4.142857
3,95680,4.111111
4,95814,4.000000
...,...,...
57,94288,3.142857
58,95823,3.133333
59,94209,3.125000
60,94254,3.125000


In [20]:
lowest_rating 

,zipcode,AvgStars
62,94230,3.0
63,94267,3.0
64,94293,3.0
65,94285,3.0
66,95624,3.0
...,...,...
121,95841,1.5
122,95671,1.5
123,94268,1.5
124,95693,1.0


#### 5.Are men or women providing the biggest share of our revenue? Create a query that lists revenue by gender. Hint: You will need purchaseprice and wholesalecost to calculate revenue.

In [21]:
mer_a = pd.merge(transaction_df, rootbeer_df, how = 'left', on = 'rootbeerid')
mer_b = pd.merge(mer_a, rootbeerbrand_df, how = 'left', on = 'brandid')
mer_c = pd.merge(mer_b, customer_df, how = 'left', on = 'customerid')
mer_c['Revenue'] = mer_c['purchaseprice'] - mer_c['wholesalecost'] 
gen_df = pd.DataFrame(mer_c, columns = ['gender', 'Revenue'])
k = (gen_df.groupby(['gender']))['Revenue'].sum()
k

gender
F    6078.91
M    5806.22
Name: Revenue, dtype: float64

In [22]:
# Women actually provide the biggest share of revenue

#### 6.How many of our customers made their first purchase in the year 2012?

In [23]:
customer_df['firstpurchaseYear'] = pd.DatetimeIndex(customer_df['firstpurchasedate']).year
purch_2012 = customer_df['firstpurchaseYear'] == 2012
print('Number of customer that made thei first purchase in 2012 are :', customer_df[purch_2012].shape[0])

Number of customer that made thei first purchase in 2012 are : 202


#### 7 Are there any root beer brands that don't have a website listed in the database?

In [24]:
 if rootbeerbrand_df['website'].isnull().sum() == 0:
    print('None')
else:
    print('Yes there are',rootbeerbrand_df['website'].isnull().sum(), 'brands without a website')

Yes there are 2 brands without a website


#### 8.Dr. Wells is very interested in how the different brands are performing. Specifically, he wants to know how many bottles (not cans) of each root beer brand have been sold in total. He only wants bottles from brands that are “currently offered” in the Root box kiosks. Create a query that lists two fields: Brand Name (which is the brand name of the Root Beer) and NumberOfBottles (which is the total number of bottles of currently offered root beer sold since the company began). Sort this list in descending order by NumberOfBottles.

In [25]:
filt_bott = mer_c['containertype'] == 'Bottle'
bottles_only = mer_c[filt_bott]
bott_Brand = pd.DataFrame(bottles_only, columns = ['brandname', 'brandid'])
f_bot = bott_Brand.groupby(['brandname'])
k= f_bot['brandid'].value_counts().sort_values(ascending = False)
New = pd.DataFrame({'Number of bottles' : k})

In [26]:
New.reset_index(inplace = True)
New

,brandname,brandid,Number of bottles
0,IBC,10016,537
1,Gales,10014,523
2,Fitz's,10012,486
3,Sparky's Fresh Draft,10013,453
4,Captain Eli's,10008,419
5,Dog n Suds,10010,410
6,Dominion,10011,394
7,Henry Weinhard's,10015,385
8,Bedfords,10005,352
9,Abita,10003,334


#### 9.A few weeks ago, Dr. Wells helped a customer who had a problem with a transaction. Dr. Wells needs to contact this customer to follow up, but unfortunately spilled all over the paper with the customer’s name and contact information. The only portion of the notes that are readable says “4509” which Dr. Wells thinks is part of a credit card number. Create a query with two fields: Name (A concatenation of the customer’s first name and last name separated by a space, e.g., “John Gregglan” but without the quotes) and Phone Number (the customer’s phone number). Hint: You will need to use the aggregate function “Group By”. Make sure not to display the sensitive credit card number.

In [27]:
mer_c['creditcardnumber'] = mer_c['creditcardnumber'].astype(str)
mer_c['Name'] = mer_c['firstname'] + " "+ mer_c['lastname']
k = mer_c['creditcardnumber'].str.contains("4509")
p = mer_c[k][['Name', 'phonenumber']].head(1)
p.set_index('Name')

,phonenumber
Name,
Ann Lane,(916) 903-7266
